In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches
import scipy
import pickle
import warnings
import bidict
import collections

import sys
sys.path.append('..')
from construct_graph.graph import Graph, Flat
import construct_graph
import solve_laplace_operator

import sparseqr

def check_convergence(L, tolerance=0.001):
    N = len(L)
    if N < 3:
        return False

    # Calculate the mean and standard deviation
    sample_mean = np.mean(L)
    sample_std = np.std(L, ddof=1)

    # Calculate the confidence interval width
    z = 1.96  # For approximately 95% confidence
    ci_half_width = z * (sample_std / np.sqrt(N))

    # Check if the mean is stable
    if N > 2:
        prev_mean = np.mean(L[:-1])
        mean_change = abs((sample_mean - prev_mean) / prev_mean)
        if mean_change > tolerance:
            return False

    # Check if confidence interval is narrow enough
    relative_ci_width = 2 * ci_half_width / sample_mean
    if relative_ci_width > tolerance:
        return False

    return True

%load_ext autoreload
%autoreload 2

In [ ]:
class Random_Geometric_Graph:

    def __init__(self, num_Vs):

        self.num_Vs = num_Vs
        self.total_V_coords, self.total_num_Vs = self.construct_total_V_coords()
        self.total_edges = self.construct_total_edges()

        self.totally_periodic = False

    def construct_total_V_coords(self):

        num_Vs = 0
        bulk_V_coords = []
        boundary_V_coords = []
        while num_Vs < self.num_Vs:
            v_coords_x, v_coords_y = np.random.uniform(-1, 2, size=(2))
            v_coords = np.array([v_coords_x, v_coords_y])
            if (0 < v_coords_x < 1) and (0 < v_coords_y < 1):
                num_Vs += 1
                bulk_V_coords.append(v_coords)
            else:
                boundary_V_coords.append(v_coords)
        
        total_V_coords = np.vstack((bulk_V_coords, boundary_V_coords))
        total_num_Vs = total_V_coords.shape[0]

        return total_V_coords, total_num_Vs
    
    def construct_total_edges(self):

        # Construct edges
        tree = scipy.spatial.cKDTree(self.total_V_coords)
        mean_l = 2.5 * 1 / np.sqrt(self.full_num_Vs)
        total_edges = np.array(list(tree.query_pairs(r=mean_l)))
        total_edges = np.sort(total_edges, axis=1)

        return total_edges